In [27]:
#uploading dataset 
from google.colab import files
uploaded = files.upload()

Saving covid.csv to covid (2).csv


In [ ]:
pip install jupyter-dash

In [31]:
#Importing necessary libraries

from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import pandas as pd
from dash.dependencies import Input, Output

#adding some external css
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

#reading file
df = pd.read_csv('covid.csv')

#DATA CLEANING

#checking for missing values in dataset
df.isnull().sum()
#replacing the values with average
df = df.fillna(df.mean())
#deleting unnecessary columns from dataset
df = df.drop(['tests_units','new_cases_smoothed','new_deaths_smoothed','new_cases_smoothed_per_million','total_deaths_per_million','new_deaths_per_million','new_deaths_smoothed_per_million','icu_patients_per_million','hosp_patients_per_million','weekly_icu_admissions_per_million','weekly_hosp_admissions_per_million','total_tests_per_thousand','new_tests_per_thousand','new_tests_per_thousand','new_tests_smoothed','new_tests_smoothed_per_thousand','tests_per_case','tests_units','people_vaccinated','new_vaccinations_smoothed','hospital_beds_per_thousand','cardiovasc_death_rate','diabetes_prevalence'], axis = 1) 
## Drop rows corresponding to the World
df = df[df.location != 'World']
## Sort df by date
df = df.sort_values(by=['date'])
# creating a new column for weeks
df['new_date'] = pd.to_datetime(df['date'])
df['Year-Week'] = df['new_date'].dt.strftime('%Y-%U')
# sorting by week
df = df.sort_values(by=['Year-Week'])
df = df.astype({"New Cases": int,"new_deaths": int,"total_cases": int,"people_fully_vaccinated":int})
all_continents = df.continent.unique()

#FRONTEND
app.layout = html.Div(children=[html.Header(children='Tracking Covid-19 Outbreak', style={'color':'#ED553B', 'font-size': 24,'font-family': 'Arial, Helvetica, sans-serif','font-weight': 'bold','text-align': 'center','background-color': 'white' }),
    html.Div([
        html.Div([
            html.Div(children='Track weekly total and new Covid-19 cases and deaths for each country', style={'color':'#173F5F', 'font-size': 18,'font-family': 'Arial, Helvetica, sans-serif','text-align': 'center','background-color': 'white' }),
            dcc.Dropdown(
              options=[{'label': 'Total Cases', 'value': 'total_cases'},
                       {'label': 'Total Deaths', 'value': 'total_deaths'},
                       {'label': 'New Deaths', 'value': 'new_deaths'},
                       {'label': 'New Cases', 'value': 'New Cases'}],
              id='ddown',
              value='total_cases',style={'background-color': '#F6D55C'}
              ), 
            dcc.Graph(id='graph1') 
        ], className='six columns',style={'background-color': '#F6D55C'}),
        html.Div([
            html.Div(children='Life expectancy Vs GDP per capita for each country during the Outbreak',style={'color':'173F5F', 'font-size': 18,'font-family': 'Arial, Helvetica, sans-serif','text-align': 'center','background-color': 'white' }),
            dcc.Checklist(
              id="checklist",
              options=[{"label": x, "value": x} 
                 for x in all_continents],
                  value=all_continents[5:],
        labelStyle={'display': 'inline-block','background-color': '#F6D55C', 'padding-left': 5,'padding-right': 22.25,'padding-bottom':6,'padding-top':6}
    ),
              dcc.Graph(id='graph2')  
        ], className='six columns'),
    ], className='row',style={'background-color': 'white'})
])

#Callbacks to update graphs

@app.callback(
  Output('graph1', 'figure'),
  Input('ddown', 'value'))
def update_figure(se):
  fig1 = px.choropleth(df, locations="iso_code", color=se, hover_name="location", projection="orthographic",labels={'Year-Week':'Year and Week', 'iso_code': 'Iso Code', 'total_cases':'Total Cases','new_deaths':'New Deaths','total_deaths':'Total Deaths'},color_continuous_scale=["#173F5F","#20639B","#3CAEA3","#F6D55C","#ED553B"],range_color=(0,25000000) ,animation_frame="Year-Week")
  fig1 = fig1.update_layout(margin={"r":0,"t":10,"l":20,"b":0},font_family="sans-serif",font_color="#173F5F", height=500)
  return fig1

@app.callback(
    Output("graph2", "figure"), 
    [Input("checklist", "value")])
def update_line_chart(continents):
    mask = df.continent.isin(continents)
    fig2 = px.scatter(df[mask], x="gdp_per_capita", y="life_expectancy",size="population", color="location", log_x=True, size_max=60, labels={'life_expectancy': 'Life Expectancy', 'gdp_per_capita': 'GDP per capita', 'population' : 'Population', 'location':'Location'}, hover_data=['location']).for_each_trace(lambda t: t.update(name=t.name.split("=")[1]))
    fig2 = fig2.update_layout(template="simple_white",margin={"r":0,"t":2,"l":0,"b":0}, font_family="sans-serif",
    font_color="#173F5F", height=500)
    return fig2



if __name__ == '__main__':
    app.run_server(debug=True)





Dash app running on:


<IPython.core.display.Javascript object>